<a href="https://colab.research.google.com/github/jamesBaker361/colab/blob/master/3_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

In [0]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

#Dataset

In [0]:
#get data!!!

path_to_file=tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [0]:
#convert her to array
text=open(path_to_file,"rb").read().decode(encoding='utf-8')
print(text[5000:5500])

or heart, the arm our soldier,
Our steed the leg, the tongue our trumpeter.
With other muniments and petty helps
In this our fabric, if that they--

MENENIUS:
What then?
'Fore me, this fellow speaks! What then? what then?

First Citizen:
Should by the cormorant belly be restrain'd,
Who is the sink o' the body,--

MENENIUS:
Well, what then?

First Citizen:
The former agents, if they did complain,
What could the belly answer?

MENENIUS:
I will tell you
If you'll bestow a small--of what you have li


In [0]:
# the unique characters that appear
vocab=sorted(set(text))
print(vocab)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [0]:
print(len(vocab))

65


In [0]:
#lets map from chars to ints
char2idx={u:i for i,u in enumerate(vocab)}
idx2char=np.array(vocab)
text_as_int=np.array([char2idx[x] for x in text])


In [0]:
print(text_as_int)
print("{}----- Characters mapped to int---> {}".format(repr(text[500:520]), text_as_int[500:520]) )

[18 47 56 ... 45  8  0]
' citizens, the patri'----- Characters mapped to int---> [ 1 41 47 58 47 64 43 52 57  6  1 58 46 43  1 54 39 58 56 47]


In [0]:
#creating learnable datasex
#hyperparamters
#using last seq_length numbers, we try to predict the seq_length+1th character

seq_length=150
examples_per_epoch=len(text)//seq_length



In [0]:
#now we need to make our data into a tensor
char_dataset=tf.data.Dataset.from_tensor_slices(text_as_int)

sequences=char_dataset.batch(seq_length+1,drop_remainder=True)

In [0]:
def split_input_target(block):
  input_text=block[:-1]
  target_text=block[1:]
  return input_text,target_text

dataset=sequences.map(split_input_target)

In [0]:
for in_example, tar_example in dataset.take(3):
  print("Input data: ",repr(''.join(idx2char[in_example.numpy()])))
  print("Target data: ",repr(''.join(idx2char[tar_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nA'
Target data:  'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAl'
Input data:  "l:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us "
Target data:  ":\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us k"
Input data:  "ill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good"
Target data:  "ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done

In [0]:
#more hyperparameters
BATCH_SIZE=250
steps_per_epoch=examples_per_epoch//BATCH_SIZE
BUFFER_SIZE=10000 
#rnns need buffer in case they turn infinite 


In [0]:
dataset=dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [0]:
dataset

<DatasetV1Adapter shapes: ((250, 150), (250, 150)), types: (tf.int64, tf.int64)>

#Model

In [0]:
#more hyperparamters
vocab_size=len(vocab) #size inout and output
embedding_dim=256 
rnn_units=1024

if tf.test.is_gpu_available():
  rnn=tf.keras.layers.CuDNNGRU #could also do CUDNNLSTM?
else:
  import functools
  rnn=functools.partial(tf.keras.layers.GRY, recurrent_activation="sigmoid")

In [0]:
def build_model(vocab_size,embedding_dim,rnn_units,batch_size):
  model=tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
      rnn(rnn_units, return_sequences=True, recurrent_initializer="glorot_uniform", stateful=True), #recurrent_initializer just initializes with less aggy weights
      tf.keras.layers.Dense(vocab_size)
  ])
  return(model)

In [0]:
model=build_model(
  vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (250, None, 256)          16640     
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (250, None, 1024)         3938304   
_________________________________________________________________
dense (Dense)                (250, None, 65)           66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


#Loss

In [0]:
def loss(labels,logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels,logits, from_logits=True)


#Training Time

In [0]:
model.compile(
optimizer=tf.train.AdamOptimizer(),
loss=loss)


In [0]:
#we gonna use some callbacks? So we can save stuff
checkpoint_dir="./training_checkpoints"
checkpoint_prefix=os.path.join(checkpoint_dir,'ckpt_{epoch}')

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
filepath=checkpoint_prefix,
save_weights_only=True)

In [0]:
history=model.fit(
dataset.repeat(),
epochs=10,
steps_per_epoch=steps_per_epoch,
callbacks=[checkpoint_callback])

Epoch 1/10
29/29 [==============================] - 19s 649ms/step - loss: 1.8481
Epoch 2/10
29/29 [==============================] - 19s 668ms/step - loss: 1.7779
Epoch 3/10
29/29 [==============================] - 19s 666ms/step - loss: 1.7155
Epoch 4/10
29/29 [==============================] - 19s 663ms/step - loss: 1.6601
Epoch 5/10
29/29 [==============================] - 19s 663ms/step - loss: 1.6097
Epoch 6/10
29/29 [==============================] - 19s 664ms/step - loss: 1.5674
Epoch 7/10
29/29 [==============================] - 19s 662ms/step - loss: 1.5297
Epoch 8/10
29/29 [==============================] - 19s 662ms/step - loss: 1.4961
Epoch 9/10
29/29 [==============================] - 19s 661ms/step - loss: 1.4650
Epoch 10/10
29/29 [==============================] - 20s 677ms/step - loss: 1.4388


#Run

In [0]:
model=build_model(vocab_size,embedding_dim,rnn_units,batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

In [0]:
def generate_text(model,start_string=" ",num_generate=1000,temperature=0.5):
  num_generate=num_generate
  
  input_eval=[char2idx[s] for s in start_string]
  input_eval=tf.expand_dims(input_eval,0)
  
  text_generated=[]
  temperature=temperature
  
  model.reset_states()
  for i in range(num_generate):
    predictions=model(input_eval)
    predictions=tf.squeeze(predictions,0)
    
    predictions=predictions/temperature
    predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
    
    input_eval=tf.expand_dims([predicted_id],0)
    text_generated.append(idx2char[predicted_id])
    
  return(start_string+"".join(text_generated))

In [0]:
print(generate_text(model,start_string=u"I Want to Fuck Juliet"))

I Want to Fuck Juliet.

CLAUDIO:
What is your noble crowns, that thou diest it do not here it
shall be some and the repting to the king and your heart,
Where is the end then he did be well of men.

CORIOLANUS:
Ay, that say. I sear my some stare
To see the matter with the first of the posest you,
Why then the prince that thou shalt bear the breasted borns and here,
What shall I would the will and man say is here.

CLARENCE:
A gentleman arred by the suncess of this face with him.

LUCIO:
I did you thou desters the world it with a grave it doth the earth to the people.

BRUTUS:
Nor I am possess, that you shall be so loth, and the senden of his head,
And praise you have heard the senden heart, even ere the born that he had fair and soul,
When all the bentle stringer to the seace to remember him place and the heart,
Which thou thou art a face of this farse and so heard
The son, the give the metter of the castless hand,
And there is the very sid and and the more than with him all the courtes

In [0]:
print(generate_text(model,start_string=u"I Want to Fuck Romeo"))

I Want to Fuck Romeo.

MARCIUS:
I have been and the norsel of his truth, by this king
And ever thou hast that do not to the head
Of the time of men and the fired than the prince
And lives the cloud of the sender them all.

CORIOLANUS:
The time that he shall be here?

CLARENCE:
What think the seas that we have then and love shall be the now all my course.

KING HENRY VI:
What is the bring of the mother words and leave to me;
Madam, to proce the county talker is a noble shall be stands that he will be senden to the corventer
Than thee in the crown of charge you than the world.

BRANUS:
The sent of the more to the princely of the princes,
And so her heaven now the highness make the love:
And the great breathed with the friend, think your head is the princely said,
And my heart the warched with the truth, but the day there he that save your sight,
The amed and all the house and the will say the world,
To be so well the companous it to the hearth,
What stand the senst thou frouse and my sou